In [1]:
import pandas as pd
import numpy as np
import os

import PivotTable as pt
import process_mangement as pm

import warnings
warnings.filterwarnings("ignore")

In [2]:
# readDb = pm.readDb(db_path)

# df_attrs_stdds = readDb.read_table('표준데이터시트_개별속성')
# df_attrs_nonstdds = readDb.read_table('비표준데이터시트_개별속성')

# df_attrs_header = readDb.read_table('속성해더')

In [3]:
# 실제 코드가 완성된 후에 앞부분은 SQLite에서 QUERY를 통해 가져와서 바로 DATAFRAME으로 변환하는 방식으로 변경할 예정
# 지금은 테스트를 위해 CSV 파일을 사용

# 데이터 불러오기
base_path_test = os.path.join(os.getcwd(), 'test_file')

df_working_f = pd.read_csv(os.path.join(base_path_test, '개별속성_작업파일(Test).csv'), dtype={'공정번호' : str}) # 작업파일
df_attrs_stdds = pd.read_csv(os.path.join(base_path_test, '개별속성_표준데이터시트(Test).csv'), dtype={'공정번호' : str}) # 표준데이터시트
df_attrs_nonstdds = pd.read_csv(os.path.join(base_path_test, '개별속성_비표준데이터시트(Test).csv'), dtype={'공정번호' : str}) # 비표준데이터시트
df_attrs_header = pd.read_csv(os.path.join(base_path_test, '속성해더.csv'), dtype={'공정번호' : str}) # 속성해더

In [4]:
df_attrs_header = df_attrs_header[df_attrs_header['속성 그룹 코드']=='01_속성명']

df_attrs_list = {'표준데이터시트' : df_attrs_stdds, '선작업데이터' : df_attrs_nonstdds}

df_fins = []

for source, df_attrs in df_attrs_list.items() :
    df_attrs_header = df_attrs_header[df_attrs_header['속성 그룹 코드']=='01_속성명']
    insert_attrs_preprocessing = pm.InsertAttrstPreprocessing(df_working_f, df_attrs_header)
    insert_attrs_preprocessing.step0_1()
    insert_attrs_preprocessing.step0_2(df_attrs, col_name=source)

    df_working_f = insert_attrs_preprocessing.df_working_f
    df_attrs = insert_attrs_preprocessing.df_attrs

    df_fin = pd.DataFrame(columns=df_attrs_header.columns)
    df_results =[]

    for i in range(len(df_attrs_header)) :
        insert_attrs_pipeline = pm.InsertAttrsPipeline(df_working_f, df_attrs, df_attrs_header)
        df_result = insert_attrs_pipeline.excute(i)
        df_results.append(insert_attrs_pipeline.df_cct_result)

    df_results.insert(0, df_fin)
    df_fin = pd.concat(df_results, axis=0)

    df_fin = pd.concat([df_fin, df_working_f], ignore_index=True)
    df_fin = df_fin.drop_duplicates(subset=['SRNo', '공종별 분류 코드'], keep='first')
    df_fin.sort_values(by=['공종별 분류 코드', '속성 그룹 코드'], inplace=True)

    df_fins.append(df_fin)

In [5]:
def merge_and_update(df1, df2) :
    """우선순위 업데이트(특정 칼럼을 키 값으로 할 경우)"""
    df_merged = pd.merge(df1, df2, on=['SRNo', '공종별 분류 코드'], suffixes=['_1st', '_2nd'], how='left')

    # update할 칼럼에 대하여 combine_first 수행
    update_columns = [f'속성{i}' for i in range(1, 8)]
    for col in update_columns :
        df_merged[col] = df_merged[f"{col}_1st"].combine_first(df_merged[f'{col}_2nd'])

    columns = [col for col in df_merged.columns.to_list() if '_1st' not in col and '_2nd' not in col]
    df_merged = df_merged[columns]
    return df_merged

In [ ]:
df_merged = merge_and_update(df_fins[0], df_fins[1])

In [ ]:
fin_result_df = pd.merge(df_working_f, df_merged, left_on=['SRNo', '공종별 분류 코드'], suffixes=['_x', ''], right_on=['SRNo', '공종별 분류 코드'], how='left')
columns = [col for col in fin_result_df.columns.to_list() if '_x' not in col]
fin_result_df = fin_result_df[columns]

In [11]:
fin_result_df.to_csv(os.path.join(base_path_test, "개별속성_작업파일_결과(Test)-result.csv"), index=False, encoding='cp949')